<a href="https://colab.research.google.com/github/karanSonker/TinyDigits/blob/main/Handwritten_Digit_Recognition_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install TensorFlow if not already installed
!pip install tensorflow

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import numpy as np


Load and Preprocess Data





In [ ]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the data
x_train, x_test = x_train / 255.0, x_test / 255.0

# Expand dimensions to match the input shape of the model
x_train = x_train[..., tf.newaxis].astype('float32')
x_test = x_test[..., tf.newaxis].astype('float32')


11490434/11490434 [==============================] - 2s 0us/step


Data Augmentation

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

# Fit the data generator to the training data
datagen.fit(x_train)


Model Definition and Training

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with data augmentation
model.fit(datagen.flow(x_train, y_train, batch_size=32),
          epochs=5,
          validation_data=(x_test, y_test))


Epoch 1/5
1875/1875 [==============================] - 39s 18ms/step - loss: 0.2509 - accuracy: 0.9205 - val_loss: 0.0457 - val_accuracy: 0.9842
Epoch 2/5
1875/1875 [==============================] - 29s 16ms/step - loss: 0.0885 - accuracy: 0.9724 - val_loss: 0.0312 - val_accuracy: 0.9899
Epoch 3/5
1875/1875 [==============================] - 24s 13ms/step - loss: 0.0644 - accuracy: 0.9798 - val_loss: 0.0365 - val_accuracy: 0.9880
Epoch 4/5
1875/1875 [==============================] - 24s 13ms/step - loss: 0.0527 - accuracy: 0.9834 - val_loss: 0.0362 - val_accuracy: 0.9885
Epoch 5/5
1875/1875 [==============================] - 24s 13ms/step - loss: 0.0489 - accuracy: 0.9847 - val_loss: 0.0285 - val_accuracy: 0.9914


 Model Quantization

In [ ]:
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Post-training quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

# Save the quantized model
with open('mnist_model_quant.tflite', 'wb') as f:
    f.write(tflite_model)


Download the Quantized Model

In [ ]:
from google.colab import files

files.download('mnist_model_quant.tflite')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 Optional - Evaluate the TFLite Model

In [ ]:
# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data
input_shape = input_details[0]['shape']

# Prepare the input data (use the test set)
input_data = np.expand_dims(x_test[0], axis=0).astype('float32')

# Set the tensor to point to the input data to be inferred
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run the interpreter
interpreter.invoke()

# Get the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])
print("Prediction results:", output_data)


Prediction results: [[3.0304710e-11 9.1184393e-09 3.3902273e-07 9.0306408e-08 7.8965767e-09
  3.4766114e-12 2.4702581e-15 9.9999905e-01 1.3488749e-09 4.3735361e-07]]
